In [1]:
from utils import find_segment_by_descriptor
from utils import load_raw_fif

raw = load_raw_fif("../../../resources/AMIGOS/processed/P01_13.fif")

/home/jacopo/PycharmProjects/progetto-tesi/common/data/eeg/mne_utils.py:12: RuntimeWarning: This filename (../../../resources/AMIGOS/processed/P01_13.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  return mne.io.read_raw_fif(path, preload=True, verbose=False)


In [4]:
o = find_segment_by_descriptor(raw, "P01_13_2")
o = o[0]

In [5]:
from utils import get_segment_raw

raw.copy().crop(tmin=o[1], tmax=o[1] + o[2])

<Raw | P01_13.fif, 17 x 1025 (8.0 s), ~156 KiB, data loaded>

In [9]:
import mne

# Fast way to only pick EEG data
picks = mne.pick_types(raw.info, eeg=True)
data = raw.get_data(picks=picks)  # shape (n_channels, n_times)
ch_names = [raw.ch_names[i] for i in picks]

In [10]:
ch_names

['AF3',
 'F7',
 'F3',
 'FC5',
 'T7',
 'P7',
 'O1',
 'O2',
 'P8',
 'T8',
 'FC6',
 'F4',
 'F8',
 'AF4']

In [11]:
data

array([[ 8.56684255e-06, -4.36757994e+00, -1.66805339e+00, ...,
         1.94080448e+01,  3.82541585e+00,  9.85738734e-06],
       [ 6.51970595e-06,  4.26448727e+00,  5.85616446e+00, ...,
         6.53107548e+00,  3.54161668e+00,  6.78007882e-06],
       [-1.05969984e-05,  5.41352463e+00,  8.66718578e+00, ...,
         1.01064844e+01,  6.35359097e+00, -1.02970034e-05],
       ...,
       [ 9.72625799e-07,  1.38734186e+00,  2.15106058e+00, ...,
         2.30761743e+00, -1.39282012e+00,  9.29230225e-07],
       [ 7.21686433e-07,  2.57303238e+00,  7.08635330e+00, ...,
        -5.65020895e+00, -3.06567740e+00,  3.87729557e-07],
       [ 5.25180167e-06,  2.05313015e+00, -1.08840141e+01, ...,
         7.27367353e+00, -1.47195446e+00,  5.94801713e-06]],
      shape=(14, 7229))

In [12]:
type(data)

numpy.ndarray